In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import os
import scipy.optimize as sc
import matplotlib as mpl
import glob
from matplotlib.path import Path
from collections import Counter
from tqdm import tqdm

In [ ]:
datapath = "/home/haue/repositories/Masters/data/" 
Al_path = datapath + "bananas/Al"
P_path = datapath + "bananas/P"
Si_path = datapath + "bananas/Si"
Mg_path = datapath + "bananas/Mg"
#outputdir = "/figure_output/"
cuts0 = datapath + "cuts/idcuts.root" 
bancuts = datapath + "cuts/banana_cuts.root"
cuts1 = datapath + "cuts/totcuts.root"

Al_files = glob.glob(os.path.join(Al_path, "*mlio.root"))
P_files = glob.glob(os.path.join(P_path, "*mlio.root"))
Si_files = glob.glob(os.path.join(Si_path, "*mlio.root"))
Mg_files = glob.glob(os.path.join(Mg_path, "*mlio.root"))

In [3]:
# Storing the data in a dictionary for easy access

### Should be a more intuitive version than the ones used in Bananachecks...

data_files = {
    "Al": Al_files,
    "P": P_files,
    "Mg": Mg_files,
    "Si": Si_files
}
data_collection = {
      "Al": {},
    "P": {},
    "Mg": {},
    "Si": {},
    "All": {}
}

data_collection = {isotope: {f"id{i}": {key: [] for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]} for i in range(4)} for isotope in data_files}
data_collection["All"] = {f"id{i}": {key: [] for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]} for i in range(4)}


for isotope, files in data_files.items():
    #file_progress = tqdm(files, desc=f"Processing {isotope}", unit="file", total=len(files), dynamic_ncols=True)
    file_progress = tqdm(files, desc=f"Processing {isotope}", unit="file", total=len(files), bar_format="{l_bar}{bar} {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")

    for file in file_progress:
        
        ## accesses the tree "a" in the root file
        r = uproot.open(file)["a"]
        ##Dummy array
        dum = r.arrays(["id", "E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"])
        
        ## converts it to an awkward array
        ids = ak.flatten(dum["id"])
        Es = ak.nan_to_num(ak.flatten(dum["E"]) / 1e3)
        fEdeps = ak.nan_to_num(ak.flatten(dum["fEdep"]) / 1e3)
        bEdeps = ak.nan_to_num(ak.flatten(dum["bEdep"]) / 1e3)
        angles = ak.nan_to_num(np.rad2deg(ak.flatten(dum["angle"])))
        Edeps = ak.nan_to_num(ak.flatten(dum["Edep"]) / 1e3)
        FIs = ak.flatten(dum["FI"])
        BIs = ak.flatten(dum["BI"])
        
        ##Creates entries in data_collection for each key and for each detector for each key
        for i in range(4):
            mask = (ids==i)
            detector_name = f"id{i}"
            data_collection[isotope][detector_name]["E"].extend(Es[mask])
            data_collection[isotope][detector_name]["fEdep"].extend(fEdeps[mask])
            data_collection[isotope][detector_name]["bEdep"].extend(bEdeps[mask])
            data_collection[isotope][detector_name]["angle"].extend(angles[mask])
            data_collection[isotope][detector_name]["Edep"].extend(Edeps[mask])
            data_collection[isotope][detector_name]["FI"].extend(FIs[mask])
            data_collection[isotope][detector_name]["BI"].extend(BIs[mask])
        file_progress.set_postfix({"Completed": f"{file_progress.n}/{file_progress.total}"})
    #print(f"Processing done for {isotope}!")
    

tqdm.write("Combining all isotopes into 'All'...")      
## Create the "All" branch in data_collection
for i in range(4):
    detector_key = f"id{i}"
    for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]:
        data_collection["All"][detector_key][key] = sum((data_collection[isotope][detector_key][key] for isotope in data_files), [])

tqdm.write("✅ All data combined successfully!")


Processing Al:   0%|           0/56 [00:00<?, ?file/s]

Processing Al: 100%|██████████ 56/56 [00:34<00:00,  1.64file/s]
Processing P: 100%|██████████ 63/63 [01:58<00:00,  1.89s/file]
Processing Mg: 100%|██████████ 12/12 [00:14<00:00,  1.18s/file]
Processing Si: 100%|██████████ 11/11 [00:28<00:00,  2.59s/file]


Combining all isotopes into 'All'...
✅ All data combined successfully!


In [2]:
E_kin = (25/57) * 30 #keV
m = 25.004109
u = 931494.10372 #keV/c^2
c = 299792458 # m/s
v = np.sqrt(2*E_kin/(m*u))*299792458
x = 0.05/v

def expdecay(N0, t_, t):
    lamb = np.log(2) / t_
    return N0-N0*np.exp((-lamb)*t)

expdecay(114123, (220.6*10**-3), (x))

0.056263905120431446

In [3]:
Mg24 = 23.985041689
Mg24_err = 0.000000014
p = 1.0078250190
p_err = 0.0000000010
frac = (Mg24/(Mg24+p))
frac_err = np.sqrt((p/(Mg24+p)**2)**2 * Mg24_err**2 + (Mg24/(p+Mg24)**2)**2 * p_err**2)
p1 = 2673.3; p2 = 4192; p3 = 4582;p4=5804;p5 = 6829;p6 = 7118;p7=7901
p1_err = 0.6; p2_err = 4; p3_err = 2;p4_err=4;p5_err = 4;p6_err = 5;p7_err=2
psep = 2271.37; g1=1368.625; g2=2754.016
psep_err = 0.06; g1_err = 0.005; g2_err = 0.011

def withg(p,p_err,gam,gam_err):
    return (p,frac*(p-psep-gam), np.sqrt((p-psep-gam)**2 * frac_err**2 + frac**2 * p_err**2 + frac**2 * psep_err**2 + frac**2 * gam_err**2))
def withoutg(p,p_err):
    return  (p,frac*(p-psep), np.sqrt((p-psep)**2 * frac_err**2 + frac**2 * p_err**2 + frac**2 * psep_err**2))
#vals = [frac*(peak1-gam-psep), frac*(peak1-psep), frac*(peak2-psep), frac*(peak2-psep-gam),frac*(5804-psep-gam), frac*(2673.3-psep), frac*(4192-psep), frac*(4582-psep), frac*(4582-psep-gam), frac*(6829-psep-g2), frac*(7646-psep)]
#sorted(vals), vals
peaks_withoutg = [withoutg(i,j) for i,j in zip([p1,p2,p3,p6,p7], [p1_err,p2_err,p3_err,p6_err,p7_err])]
peaks_withg1 = [withg(i,j,g1,g1_err) for i,j in zip([p3,p4,p6,p7],[p3_err,p4_err,p6_err,p7_err])]
peaks_withg2 = withg(p5,p5_err,g2,g2_err)
peaks_withoutg, peaks_withg1, peaks_withg2, frac
#sorted(vals)

([(2673.3, 385.72237105453763, 0.5786771607068808),
  (4192, 1843.1815428519299, 3.8391338032060935),
  (4582, 2217.4549852704345, 1.9202145004587217),
  (7118, 4651.192036484095, 4.798722937645029),
  (7901, 5402.617947801247, 1.9202145004587354)],
 [(4582, 904.0191131421628, 1.920220495723927),
  (5804, 2076.7425660534773, 3.8391368018535106),
  (7118, 3337.756164355824, 4.798725336660437),
  (7901, 4089.1820756729758, 1.920220495723936)],
 (6829, 1730.8841553184843, 3.8391483166378255),
 0.9596754933807812)

In [4]:
Ne20 = 19.9924401753
p = 1.0078250190
frac = (Ne20/(p+Ne20))
peak1 = 8976; peak2 = 4468; peak3 = 4294; psep = 2431; twoplus = 4065
(peak1-psep)*frac,(peak1-twoplus)*frac, (peak3-psep)*frac, (peak2-psep)*frac, (5380-psep-4065+2431)*frac


(6230.8985023129435,
 4675.315896846274,
 1773.5926523772366,
 1939.242207671729,
 1251.8917540934333)

In [3]:
Al25 = 24.99042831
frac = (Al25/(p+Al25))
peak = 13015; psep= 5513.8; gam=970
(peak-psep)*frac, (peak-psep-gam)*frac-6363

(7210.41519469579, -84.98265349582107)

In [4]:
import numpy as np
Na21 =20.99765446
frac = (Na21/(p+Na21))
id3 = np.array([688.3, 1247.7, 2488.1, 3885.3, 5464.7, 6522.5])
idless = np.array([691.7, 1250.8, 2476.8, 3885.3, 5464.6, 6526.4]) # much wider peaks
frac


0.9542011788490329